this notebook contain steps about how I clean the data
a bit amburadul actually

In [2]:
import duckdb

# Create DB Connection

In [5]:
con = duckdb.connect('entity.db')

# CSV to Duckdb

In [6]:
import pandas as pd

df = pd.read_csv('entity.csv',dtype=str)
df_head = df.head()

# create the table "my_table" from the DataFrame "my_df"
con.sql("CREATE TABLE entity AS SELECT * FROM df_head")

# insert into the table "my_table" from the DataFrame "my_df"
con.sql("INSERT INTO entity SELECT * FROM df")

In [7]:
con.sql("SELECT * FROM entity LIMIT 3")

┌──────────┬──────────────────┬─────────────┬──────────────┬─────────┬────────────┬──────────┬───────────┬────────────┐
│   nama   │       nik        │ tempatLahir │ jenisKelamin │   tps   │  provinsi  │ kab_kota │ kecamatan │    desa    │
│ varchar  │     varchar      │   varchar   │   varchar    │ varchar │  varchar   │ varchar  │  varchar  │  varchar   │
├──────────┼──────────────────┼─────────────┼──────────────┼─────────┼────────────┼──────────┼───────────┼────────────┤
│ AI POPON │ 320416540758**** │ BANDUNG     │ P            │ 1       │ JAWA BARAT │ BANDUNG  │ ARJASARI  │ ANCOLMEKAR │
│ KOMALA   │ 320416440470**** │ BANDUNG     │ P            │ 1       │ JAWA BARAT │ BANDUNG  │ ARJASARI  │ ANCOLMEKAR │
│ WATI,    │ 320416410784**** │ BANDUNG     │ P            │ 1       │ JAWA BARAT │ BANDUNG  │ ARJASARI  │ ANCOLMEKAR │
└──────────┴──────────────────┴─────────────┴──────────────┴─────────┴────────────┴──────────┴───────────┴────────────┘

# Create UDF

In [195]:
from datetime import date, datetime, timedelta

# sundanese name
def find_sunda_name(name:str) -> str:
    sunda_name = "Aa Adang Ade Agah Agan Ace Aceng Aep Aam Aang Adeng Adun Aep Ageng Agus Ajang Ajat Aom Asep Atang Atep Ayi Barna Bagja Ceceng Cecep Cahya Cucu Cece Dedi Dadan Dadang Dayat Didin Deden Dana Dedeng Edi Eep Ece Ence Engkus Enjang Encep Endang Engkos Entang Galih Gilang Gumelar Ginanjar Gugu Gugun Hamdan Hendra Henhen Harja Idang Idi Ijang Ihin Jalu Jaka Jajang Jejen Junaedi Kamandaka Karta Kosasih Koswara Kurniawan Kertarajasa Maman Markasih Memed Nandang Nataprawira Oded Pepen Pradana Prawiraharja Raka Rukmana Sanjaya Senjaya Sobarna Solihin Sudarsana Suganda Suhendar Sujoni Supriatna Sutrisna Sutresna Surawijaya Tisna Udin Uu Wisesa Wawan Yana Yayat".lower().split(' ')
    for n in name.lower().split(' '):
        if n in sunda_name:
            return n
    raise ValueError("ERROR")

# javanese name
def find_java_name(name:str) -> str:
    for n in name.lower().split(' '):
        if n[-1] == 'o':
            return n
    raise ValueError("ERROR")

# bataknese name
def find_batak_name(name:str) -> str:
    batak_name = "Ajartambun,Ambarita,Aritonang,Aruan,Banjarnahor,Bakkara,Beras,Buaton,Babo,Bako,Bancin,Banuarea,Baruara,Barus,Batubara,Benjerang,Berampu,Berasa,Beringin,Bintang,Bondar,Bondong,Brahmana,Bukit,Buniaji,Busuk,Capah,Colia,Dalimunthe,Damanik,Dasopang,Daulay,Debataraja,Depari,Doloksaribu,Dongoran,Gajah,Ganagana,Garamata,Gerneng,Ginting,Girsang,Gersang,Gorat,Gultom,Gurning,Gurukinayan,Gurupatih,Gurusinga,Habeahan,Harianja,Harahap,Haro,Haromunthe,Hasibuan,Hasugian,Hutabalian,Hutabarat,Hutagalung,Hutagaol,Hutahaean,Hutajulu,Hutasoit,Hutapea,Hutasuhut,Hutauruk,Jadibata,Jampang,Jawak,Kaban,Kacaribu,Karo-karo,Keling,Kembaren,Kemit,Ketaren,Limbong,Lingga,Lubis,Lumbanbatu,Lumbangaol,Lumbannahor,Lumbanraja,Lumbansiantar,Lumbantoruan,Lumbantobing,Lumbantungkup,Maha,Malau,Manalu,Manihuruk,Manik,Simanullang,Manurung,Mardia,Marpaung,Matondang,Milala,Meliala,Muham,Munte,Munthe,Submarga Ginting,Submarga Saragih,Nababan,Nadapdap,Nadeak,Nahampun,Naibaho,Naiborhu,Nainggolan,Naipospos,Napitu,Napitupulu,Nasution,Ompusunggu,Padang,Padang Batanghari,Pakpahan,Pandiangan,Pandebayang,Pane,Pangaribuan,Panggabean,Panjaitan,Parapat,Pardede,Pardosi,Parhusip,Paroka,Parinduri,Pasaribu,Pase,Pekan,Pelawi,Penarik Pinayungan,Perangin-angin,Pohan,Pulungan,Purba,Rambe,Rajagukguk,Rangkuti,Ritonga,Rumabutar,Rumagorga,Rumahole,Rumahorbo,Rumaijuk,Rumapea,Rumasingap,Rumasondi,Sagala,Samosir,Samura,Saragi,Saragih,Seragih,Saruksuk,Sarumpaet,Sekali,Sembiring,Siadari,Siagian,Siahaan,Siallagan,Sianipar,Sianturi,Sibarani,Sibagariang,Sinagabariang,Siboro,Sibero,Cibro,Sibuea,Siburian,Sidauruk,Sidabalok,Sidabariba,Sinabariba,Sidabutar,Sidabungke,Sigalingging,Sihaloho,Haloho,Keloko,Sidari,Sidasuha,Dasuha,Sigukguhi,Sihite,Sihole,Sihombing,Sihotang,Siketang,Sijabat,Silaen,Silaban,Silalahi,Sinulaki,Silangit,Silitonga,Simaibang,Simalango,Simamora,Simandalahi,Simangunsong,Simanjorang,Simanjuntak,Simanungkalit,Simaremare,Simargolang,Simarmata,Simatupang,Simbolon,Simbulan,Simorangkir,Sinaga,Sinambela,Sinisuka,Sinubulan,Sinuhaji,Sinukaban,Sinukapur,Sinulingga,Sinupayung,Sinuraya,Sinusinga,Sinurat,Sipahutar,Sipangkar,Sipayung,Sirait,Siregar,Siringoringo,Sitepu,Sitindaon,Sitinjak,Sitio,Sitohang,Sitompul,Sitorus,Situmeang,Situmorang,Situngkir,Sorimunggu,Sormin,Sugihen,Suka,Sumbayak,Surbakti,Tampubolon,Tamba,Tambak,Tanjung,Tambun,Tambunan,Tampune,Tarigan,Tarihoran,Tegur,Tekang,Tinambunan,Togatorop,Tondang,Tendang,Torong,Tua,Tumanggor,Turnip,Ujung"
    batak_name = batak_name.lower().split(',')
    for n in name.lower().split(' '):
        if n in batak_name:
            return n
    raise ValueError("ERROR")

# signs
def find_signs(nik:str) -> str:
    #320416540758****
    #32 prov
    #04 kota kab
    #16 kec
    #54 hari wanita - 40
    #07 bulan
    #58 tahun
    tgl = int(nik[6:8])
    tgl = tgl - 40 if tgl > 40 else tgl
    bulan = int(nik[8:10])
    signs = {
        'aries':(date(2000,3,21),date(2000,4,20)),
        'taurus':(date(2000,4,20),date(2000,5,21)),
        'gemini':(date(2000,5,21),date(2000,6,21)),
        'cancer':(date(2000,6,21),date(2000,7,23)),
        'leo':(date(2000,7,23),date(2000,8,23)),
        'virgo':(date(2000,8,23),date(2000,9,23)),
        'libra':(date(2000,9,23),date(2000,10,23)),
        'scorpio':(date(2000,10,23),date(2000,11,22)),
        'sagittarius':(date(2000,11,23),date(2000,12,22)),
        'capricorn':(date(2000,12,22),date(2000,1,20)),
        'aquarius':(date(2000,1,20),date(2000,2,19)),
        'pisces':(date(2000,2,19),date(2000,3,21)),
    }
    birth = date(2000,bulan,tgl)
    for sign in signs:
        if signs[sign][0] <= birth and signs[sign][1] >= birth:
            return sign
    raise ValueError("ERROR")

# r initial
def find_r_initial(name:str) -> str:
    for n in name.lower().split(' '):
        if n[0] == 'r':
            return n
    raise ValueError("ERROR")

# christian name
def find_christian_name(name:str) -> str:
    christian_names = 'Mary,Wartono,Kristina,Zara,Lidia,Darius,Jane,Antonius,Diana,Kris,Iskandar,Florens,Nikodemus,Christ,Leonard,Novita,Carolus,Nus,Ivan,Jacobus,Quintinus,Elisa,Joice,Emmanuel,Lukas,Zainab,Petrus,Eve,Marius,Quintus,Maurits,Paulus,Indra,Umar,Hana,Xaverius,Irena,Denny,Ruth,Agnes,Jeremiah,Ferdinand,Augustinus,Isaac,Vanessa,Jose,Heliana,Natalia,Jacob,Zulfa,Robertus,Teddy,Luke,Delima,Ferry,Octavia,Susanna,Gabriel,Hendra,Julia,Alex,Josef,Renata,Aaron,Veronika,Mariana,Naomi,Vincentius,Cornelius,Magdalena,Andreas,Etika,Benjamin,Theodora,Imelda,Hani,Xavier,Clara,Joseph,Gunawan,Quinten,Sarah,Zachariah,Yohanes,Ester,Theresia,Niko,Thomas,Suzan,Winda,Bertha,Noah,Bernardus,Caleb,Rachel,Emilia,Angelique,Gregorius,Bernadette,Martinus,Elizabeth,Isaiah,David,Maria,Yuni,Jusuf,Uriah,Rafael,Miriam,Zebulun,Laurentius,Edith,Esther,Joshua,Alvin,Timothy,Karmel,Armand,Joel,Agustinus,Bimo,Gideon,Lorens,Ignatius,Monica,Judith,Lisa,Katharina,Yos,Brigitta,Melinda,Cindy,Fabian,Yosua,Solomon,Fransiskus,Tommy,Nathan,Yolanda,Lucia,Matthew,Grace,Adam,Hannah,Franciska,Leticia,Ursula,Priscilla,Zephaniah,Daniel,Ronald,Xenia,Endah,Markus,Oktavianus,Vera,Philipus,Wenny,Julius,Karmen,Rosalina,Willy,Margareta,Samuel,Leah,Feliks,Herlina,Ezekiel,James,Philip,Tobias,Jozua,Henny,Abigail,Nehemiah,Hendrik,George,Kristoforus,Janice,Jonah,Qonita,Gerry,Agatha,Peter,Jessica,Dominikus,Stephen,Stefanus,Christina,Jason,Andre,Elisabeth,Anastasia,Martha,Bobby,Simon,Matius,Joanna,Yunita,Rebecca,Katarina,Angelina,Deborah,Tania,Olivia,Paula,Gerda,Vania,Stella,Leni,John,Johannes,Hendrikus,Peni,Albert,Hermina,Paul,Felicia,Frans,Dina,Lydia,Irawan,Angela,Zefanya,Moses,Michael,Mathilda,Jakobus,Gita,Laura,Evan,Gabriella,Edwin,Benyamin,Filipus,Zakaria,Yobel,Merry'.lower().split(',')
    for n in name.lower().split(' '):
        if n in christian_names:
            return n
    raise ValueError("ERROR")

# find age
def find_age(nik:str) -> int:
    tahun = find_birth_year(nik)
    age = datetime.now().date().year - tahun
    return age

# find birth year
def find_birth_year(nik:str) -> int:
    tahun = int(nik[10:12])
    return tahun

# find tanggal produksi
def find_production_date(nik:str) -> date:
    tgl = int(nik[6:8])
    tgl = tgl - 40 if tgl > 40 else tgl
    bulan = int(nik[8:10])
    tahun = int(nik[10:12]) + 1900
    birth_date = date(year=tahun,month=bulan,day=tgl)
    production_date = birth_date - timedelta(days=(30*9)+10)
    
    return production_date

# find hari produksi
def find_production_day(production_date:date) -> str:
    return production_date.strftime("%A")


# dareah pendatang
def find_newcomer(nik:str) -> str:
    if nik[0:2] != '32':
        return 'pendatang'
    raise ValueError("ERROR")

def create_unique_location(prov,kabkota,kec,desa):
    prov = str(prov)
    kabkota = str(kabkota).replace('KOTA ','').replace('KABUPATEN ','')
    kec = str(kec).replace('KECAMATAN ','')
    desa = str(desa).replace('DESA ','')
    return '_'.join([prov,kabkota,kec,desa])



'''
- asep V
- batak V
- astrologi V
- all name V
- cewe cowo V
- inisial r V
- cewe nama V
- jawa V
- nama kristen V
- produksi V
- trend nama V
- daerah pendatang V
- pergerakan
'''


'\n- asep V\n- batak V\n- astrologi V\n- all name V\n- cewe cowo V\n- inisial r V\n- cewe nama V\n- jawa V\n- nama kristen V\n- produksi V\n- trend nama V\n- daerah pendatang V\n- pergerakan\n'

In [196]:
con.remove_function('find_newcomer')
con.remove_function('find_sunda_name')
con.remove_function('find_java_name')
con.remove_function('find_batak_name')
con.remove_function('find_signs')
con.remove_function('find_r_initial')
con.remove_function('find_christian_name')
con.remove_function('find_age')
con.remove_function('find_birth_year')
con.remove_function('find_production_date')
con.remove_function('find_production_day')
con.remove_function('create_unique_location')

from duckdb.typing import VARCHAR, DATE, INTEGER

con.create_function('find_newcomer',find_newcomer,[VARCHAR],VARCHAR,exception_handling='return_null')
con.create_function('find_sunda_name',find_sunda_name,[VARCHAR],VARCHAR,exception_handling='return_null')
con.create_function('find_java_name',find_java_name,[VARCHAR],VARCHAR,exception_handling='return_null')
con.create_function('find_batak_name',find_batak_name,[VARCHAR],VARCHAR,exception_handling='return_null')
con.create_function('find_signs',find_signs,[VARCHAR],VARCHAR,exception_handling='return_null')
con.create_function('find_r_initial',find_r_initial,[VARCHAR],VARCHAR,exception_handling='return_null')
con.create_function('find_christian_name',find_christian_name,[VARCHAR],VARCHAR,exception_handling='return_null')
con.create_function('find_age',find_age,[VARCHAR],INTEGER,exception_handling='return_null')
con.create_function('find_birth_year',find_birth_year,[VARCHAR],INTEGER,exception_handling='return_null')
con.create_function('find_production_date',find_production_date,[VARCHAR],DATE,exception_handling='return_null')
con.create_function('find_production_day',find_production_day,[DATE],VARCHAR,exception_handling='return_null')
con.create_function('create_unique_location',create_unique_location,[VARCHAR,VARCHAR,VARCHAR,VARCHAR],VARCHAR,exception_handling='return_null')

In [45]:
con.sql("SELECT find_production_date(nik),nik,nama,jenisKelamin FROM entity LIMIT 3")

┌───────────────────────────┬──────────────────┬──────────┬──────────────┐
│ find_production_date(nik) │       nik        │   nama   │ jenisKelamin │
│           date            │     varchar      │ varchar  │   varchar    │
├───────────────────────────┼──────────────────┼──────────┼──────────────┤
│ 1957-10-07                │ 320416540758**** │ AI POPON │ P            │
│ 1969-06-28                │ 320416440470**** │ KOMALA   │ P            │
│ 1983-09-25                │ 320416410784**** │ WATI,    │ P            │
└───────────────────────────┴──────────────────┴──────────┴──────────────┘

# Run UDF

In [115]:
con.sql("ALTER TABLE entity ADD COLUMN unique_location VARCHAR")

In [48]:
# con.sql("ALTER TABLE entity ADD COLUMN newcomer VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN nama_sunda VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN nama_jawa VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN nama_batak VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN bintang VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN inisial_r VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN nama_kristen VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN umur VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN tahun_lahir VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN tanggal_produksi VARCHAR")
# con.sql("ALTER TABLE entity ADD COLUMN hari_produksi VARCHAR")

con.sql("ALTER TABLE entity ALTER COLUMN umur SET DATA TYPE INTEGER;")
con.sql("ALTER TABLE entity ALTER COLUMN tahun_lahir SET DATA TYPE INTEGER;")
con.sql("ALTER TABLE entity ALTER COLUMN tanggal_produksi SET DATA TYPE DATE;")

In [141]:
con.sql("UPDATE entity SET unique_location = create_unique_location(provinsi, kab_kota, kecamatan, desa)")

In [49]:
con.sql("UPDATE entity SET newcomer = find_newcomer(nik)")
con.sql("UPDATE entity SET nama_sunda = find_sunda_name(nama)")
con.sql("UPDATE entity SET nama_jawa = find_java_name(nama)")
con.sql("UPDATE entity SET nama_batak = find_batak_name(nama)")
con.sql("UPDATE entity SET bintang = find_signs(nik)")
con.sql("UPDATE entity SET inisial_r = find_r_initial(nama)")
con.sql("UPDATE entity SET nama_kristen = find_christian_name(nama)")
con.sql("UPDATE entity SET umur = find_age(nik)")
con.sql("UPDATE entity SET tahun_lahir = find_birth_year(nik)")
con.sql("UPDATE entity SET tanggal_produksi = find_production_date(nik)")
con.sql("UPDATE entity SET hari_produksi = find_production_day(tanggal_produksi)")

con.sql("SELECT * FROM entity LIMIT 3")

┌──────────┬──────────────────┬─────────────┬──────────────┬───┬─────────────┬──────────────────┬───────────────┐
│   nama   │       nik        │ tempatLahir │ jenisKelamin │ … │ tahun_lahir │ tanggal_produksi │ hari_produksi │
│ varchar  │     varchar      │   varchar   │   varchar    │   │    int32    │       date       │    varchar    │
├──────────┼──────────────────┼─────────────┼──────────────┼───┼─────────────┼──────────────────┼───────────────┤
│ AI POPON │ 320416540758**** │ BANDUNG     │ P            │ … │          58 │ 1957-10-07       │ Monday        │
│ KOMALA   │ 320416440470**** │ BANDUNG     │ P            │ … │          70 │ 1969-06-28       │ Saturday      │
│ WATI,    │ 320416410784**** │ BANDUNG     │ P            │ … │          84 │ 1983-09-25       │ Sunday        │
├──────────┴──────────────────┴─────────────┴──────────────┴───┴─────────────┴──────────────────┴───────────────┤
│ 3 rows                                                                                

In [142]:
con.sql("SELECT unique_location FROM entity LIMIT 3")

┌────────────────────────────────────────┐
│            unique_location             │
│                varchar                 │
├────────────────────────────────────────┤
│ JAWA BARAT_BANDUNG_ARJASARI_ANCOLMEKAR │
│ JAWA BARAT_BANDUNG_ARJASARI_ANCOLMEKAR │
│ JAWA BARAT_BANDUNG_ARJASARI_ANCOLMEKAR │
└────────────────────────────────────────┘

In [26]:
find_newcomer('320416540758****')

''

# Append Coordinate Data of Desa/Kelurahan

In [75]:
coordinate = pd.read_csv('dpmdes-idm_dftr_titik_koordinat_desa__des_kel_data.csv',dtype=str )
print(len(coordinate))
coordinate.dropna(inplace=True)
print(len(coordinate))
coordinate.head(10)


10624
10622


,kode_provinsi,nama_provinsi,bps_kode_kabupaten_kota,bps_nama_kabupaten_kota,bps_kode_kecamatan,bps_nama_kecamatan,bps_kode_desa_kelurahan,bps_nama_desa_kelurahan,kemendagri_kode_kecamatan,kemendagri_nama_kecamatan,kemendagri_kode_desa_kelurahan,kemendagri_nama_desa_kelurahan,titik_longitude,titik_latitude,tahun,id
0,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190006.0,WANAHERANG,32.01.02,GUNUNG PUTRI,32.01.02.2001,WANAHERANG,-6419667.0,106.943229,2019.0,1
1,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190010.0,BOJONG KULUR,32.01.02,GUNUNG PUTRI,32.01.02.2002,BOJONG KULUR,1069697.0,-632325.0,2019.0,2
2,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190009.0,CIANGSANA,32.01.02,GUNUNG PUTRI,32.01.02.2003,CIANGSANA,106956901.0,-6371751.0,2019.0,3
3,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190002.0,GUNUNG PUTRI,32.01.02,GUNUNG PUTRI,32.01.02.2004,GUNUNG PUTRI,6.32325,106.9697,2019.0,4
4,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190004.0,BOJONG NANGKA,32.01.02,GUNUNG PUTRI,32.01.02.2005,BOJONG NANGKA,106897198.0,6430901.0,2019.0,5
5,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190003.0,TLAJUNG UDIK,32.01.02,GUNUNG PUTRI,32.01.02.2006,TLAJUNG UDIK,106.67361,-6.61083,2019.0,6
6,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190005.0,CICADAS,32.01.02,GUNUNG PUTRI,32.01.02.2007,CICADAS,-70.476551,106.328055,2019.0,7
7,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190007.0,CIKEAS UDIK,32.01.02,GUNUNG PUTRI,32.01.02.2008,CIKEAS UDIK,1065551.0,-62358.0,2019.0,8
8,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190008.0,NAGRAK,32.01.02,GUNUNG PUTRI,32.01.02.2009,NAGRAK,106.944714 BT,-63.846283 LS,2019.0,9
9,32.0,JAWA BARAT,3201.0,KABUPATEN BOGOR,3201190.0,GUNUNG PUTRI,3201190001.0,KARANGGAN,32.01.02,GUNUNG PUTRI,32.01.02.2010,KARANGGAN,106.88519,-6.456611,2019.0,10


In [95]:
import re
def preproces_koordinat(row) -> str:
    lat:str = row['titik_latitude']
    long:str = row['titik_longitude']
    try:
        if not re.sub(r'[^\d]' , '', lat):
            return ','

        lat = lat.replace("'","").replace(".","")
        long = long.replace("'","").replace(".","")
        
        lat = ''.join(c for c in lat if c.isdigit() or c == '-')
        long = ''.join(c for c in long if c.isdigit() or c == '-')

        temp = long
        if lat[0:2] == '10':
            long = lat
            lat = temp
        
        long = long[0:3] + '.' + long[3:]
        
        if lat[0] != '-':
            lat = '-'+lat
        lat = lat[0:2] + '.' + lat[2:]
    except Exception as e:
        print(e)
        print(row)
    return lat + ',' +long


In [97]:
coordinate['latlong'] = coordinate.apply(preproces_koordinat,axis=1)
coordinate['lat'] = coordinate.latlong.apply(lambda x: x.split(',')[0])
coordinate['long'] = coordinate.latlong.apply(lambda x: x.split(',')[1])
coordinate[['titik_latitude','titik_longitude','latlong','lat','long']]

,titik_latitude,titik_longitude,latlong,lat,long
0,106.943229,-6419667.0,"-6.4196670,106.943229",-6.4196670,106.943229
1,-632325.0,1069697.0,"-6.323250,106.96970",-6.323250,106.96970
2,-6371751.0,106956901.0,"-6.3717510,106.9569010",-6.3717510,106.9569010
3,106.9697,6.32325,"-6.32325,106.9697",-6.32325,106.9697
4,6430901.0,106897198.0,"-6.4309010,106.8971980",-6.4309010,106.8971980
...,...,...,...,...,...
10619,1083706.0,72209.0,"-7.22090,108.37060",-7.22090,108.37060
10620,7353968.0,108616005.0,"-7.3539680,108.6160050",-7.3539680,108.6160050
10621,72221.0,1083737.0,"-7.22210,108.37370",-7.22210,108.37370
10622,-7.362473,108.637512,"-7.362473,108.637512",-7.362473,108.637512


In [126]:
coordinate['kab_kota'] = coordinate['bps_nama_kabupaten_kota'].apply(
    lambda x: x.replace('KABUPATEN ','').replace('KOTA ','')
)
coordinate['unique_hash'] = (
    coordinate['nama_provinsi'] + '_' +
    coordinate['kab_kota'] + '_' +
    coordinate['bps_nama_kecamatan'] + '_' +
    coordinate['bps_nama_desa_kelurahan']
)
coordinate[['unique_hash','lat','long']].unique_hash.value_counts().sort_values()
coor = coordinate[['unique_hash','lat','long']].groupby(['unique_hash','lat','long']).value_counts().reset_index()
coor

,unique_hash,lat,long,count
0,JAWA BARAT_BANDUNG BARAT_BATUJAJAR_BATUJAJAR B...,-6.91949,107.49258,1
1,JAWA BARAT_BANDUNG BARAT_BATUJAJAR_BATUJAJAR B...,-6.91949,107.492580,1
2,JAWA BARAT_BANDUNG BARAT_BATUJAJAR_BATUJAJAR T...,-6.913388,107.944593,1
3,JAWA BARAT_BANDUNG BARAT_BATUJAJAR_BATUJAJAR T...,-6.9133880,107.9445930,1
4,JAWA BARAT_BANDUNG BARAT_BATUJAJAR_CANGKORAH,-6.5337,107.292,2
...,...,...,...,...
6476,JAWA BARAT_TASIKMALAYA_TARAJU_PAGERALAM,-7.4330151,107.963187314,2
6477,JAWA BARAT_TASIKMALAYA_TARAJU_PURWARAHAYU,-7.25175,107.5948,2
6478,JAWA BARAT_TASIKMALAYA_TARAJU_RAKSASARI,-7.27345,108.5912,2
6479,JAWA BARAT_TASIKMALAYA_TARAJU_SINGASARI,-7.27245,107.00,2


In [138]:
coor[coor['unique_hash'].str.contains('KABUPATEN ')]

,unique_hash,lat,long,count


In [119]:
coor[(coor['unique_hash']=='JAWA BARAT_BANDUNG_ARJASARI_ANCOLMEKAR')]

,unique_hash,lat,long
1231,JAWA BARAT_BANDUNG_ARJASARI_ANCOLMEKAR,-7.57,107.4037
6543,JAWA BARAT_BANDUNG_ARJASARI_ANCOLMEKAR,-7.57,107.4037


In [127]:
con.register('coordinate', coor)

In [128]:
con.sql("SELECT * FROM coordinate LIMIT 3")

┌────────────────────────────────────────────────────┬───────────┬────────────┬───────┐
│                    unique_hash                     │    lat    │    long    │ count │
│                      varchar                       │  varchar  │  varchar   │ int64 │
├────────────────────────────────────────────────────┼───────────┼────────────┼───────┤
│ JAWA BARAT_BANDUNG BARAT_BATUJAJAR_BATUJAJAR BARAT │ -6.91949  │ 107.49258  │     1 │
│ JAWA BARAT_BANDUNG BARAT_BATUJAJAR_BATUJAJAR BARAT │ -6.91949  │ 107.492580 │     1 │
│ JAWA BARAT_BANDUNG BARAT_BATUJAJAR_BATUJAJAR TIMUR │ -6.913388 │ 107.944593 │     1 │
└────────────────────────────────────────────────────┴───────────┴────────────┴───────┘

In [129]:
con.sql("SELECT * FROM entity WHERE desa='BODESARI' LIMIT 3")

┌─────────────────┬──────────────────┬─────────────┬───┬──────────────────┬───────────────┬──────────────────────┐
│      nama       │       nik        │ tempatLahir │ … │ tanggal_produksi │ hari_produksi │   unique_location    │
│     varchar     │     varchar      │   varchar   │   │       date       │    varchar    │       varchar        │
├─────────────────┼──────────────────┼─────────────┼───┼──────────────────┼───────────────┼──────────────────────┤
│ JAMILAH         │ 320918701055**** │ CIREBON     │ … │ 1955-01-23       │ Sunday        │ JAWA BARAT_CIREBON…  │
│ FITRIA KHASANAH │ 320918620395**** │ CIREBON     │ … │ 1994-06-15       │ Wednesday     │ JAWA BARAT_CIREBON…  │
│ RATMINI         │ 320918560364**** │ CIREBON     │ … │ 1963-06-10       │ Monday        │ JAWA BARAT_CIREBON…  │
├─────────────────┴──────────────────┴─────────────┴───┴──────────────────┴───────────────┴──────────────────────┤
│ 3 rows                                                                        

# Join Coordinate Data and Entity

In [143]:
con.sql('DROP TABLE IF EXISTS entity_with_coordinate')
con.sql(
    """
    CREATE TABLE entity_with_coordinate AS
    SELECT * FROM entity LEFT JOIN coordinate 
    ON entity.unique_location = coordinate.unique_hash
    """
    )

In [144]:
no_coor_df = con.sql('SELECT unique_location, COUNT(nik) FROM entity_with_coordinate WHERE lat IS NULL GROUP BY unique_location').fetchdf()
no_coor_df.head()

,unique_location,count(nik)
0,JAWA BARAT_INDRAMAYU_TERISI_CIBERENG,4426
1,JAWA BARAT_INDRAMAYU_TERISI_CIKAWUNG,5116
2,JAWA BARAT_INDRAMAYU_TERISI_JATIMULYA,6208
3,JAWA BARAT_INDRAMAYU_TERISI_JATIMUNGGUL,4321
4,JAWA BARAT_INDRAMAYU_TERISI_KARANGASEM,4226


In [160]:
no_coor_df

,unique_location,count(nik)
0,JAWA BARAT_INDRAMAYU_TERISI_CIBERENG,4426
1,JAWA BARAT_INDRAMAYU_TERISI_CIKAWUNG,5116
2,JAWA BARAT_INDRAMAYU_TERISI_JATIMULYA,6208
3,JAWA BARAT_INDRAMAYU_TERISI_JATIMUNGGUL,4321
4,JAWA BARAT_INDRAMAYU_TERISI_KARANGASEM,4226
...,...,...
749,JAWA BARAT_BANDUNG_BANDUNG WETAN_CIHAPIT,2998
750,JAWA BARAT_BANDUNG_CIBEUNYING KALER_CIHAUR GEULIS,8854
751,JAWA BARAT_BANDUNG_CIBEUNYING KALER_SUKALUYU,11147
752,JAWA BARAT_BANDUNG_GEDEBAGE_CIMINCRANG,2561


# Find Missing Data With Scraping

In [165]:
import requests
import time

def scrap_coordinate(unique_location:str):
    time.sleep(3)
    try:
        ul = unique_location.split('_')
        desa = ul[-1].lower()
        kec = ul[-2].lower()
        kota = ul[-3].lower()
        prov = ul[-4].lower()
        url = f'https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/findAddressCandidates?SingleLine={desa}, {kec}, {kota}, {prov}, IDN&f=json&outFields=*&countryCode=IDN&distance=50000&location=%7B%22x%22%3A12514755.34118177%2C%22y%22%3A-914900.4495666418%2C%22spatialReference%22%3A%7B%22wkid%22%3A102100%7D%7D&maxLocations=1'
        # JAWA BARAT_INDRAMAYU_TERISI_CIBERENG
        data = requests.get(url).json()

        location = data['candidates'][0]['location']
        long = location['x']
        lat = location['y']
        
        address = data['candidates'][0]['address']
        return lat,long,address
    except Exception as e:
        print(unique_location,e)



In [166]:
scrap_coordinate('JAWA BARAT_BANDUNG_GEDEBAGE_CISARANTEUN KIDUL')

(-6.948949999999968,
 107.69291000000004,
 'Cisaranten Kidul, Gedebage, Bandung, Jawa Barat')

In [167]:
no_coor_df['koordinat'] = no_coor_df['unique_location'].apply(scrap_coordinate)

In [169]:
no_coor_df.to_csv('scrap_coordinate.csv',index=False)

In [173]:
no_coor_df['lat'] = no_coor_df['koordinat'].apply(lambda x: x[0])
no_coor_df['long'] = no_coor_df['koordinat'].apply(lambda x: x[1])
no_coor_df[['unique_location','lat','long']]

,unique_location,lat,long
0,JAWA BARAT_INDRAMAYU_TERISI_CIBERENG,-6.45484,108.16669
1,JAWA BARAT_INDRAMAYU_TERISI_CIKAWUNG,-6.60706,107.98109
2,JAWA BARAT_INDRAMAYU_TERISI_JATIMULYA,-6.50137,108.14761
3,JAWA BARAT_INDRAMAYU_TERISI_JATIMUNGGUL,-6.53789,108.11278
4,JAWA BARAT_INDRAMAYU_TERISI_KARANGASEM,-6.47103,108.17059
...,...,...,...
749,JAWA BARAT_BANDUNG_BANDUNG WETAN_CIHAPIT,-6.90957,107.62559
750,JAWA BARAT_BANDUNG_CIBEUNYING KALER_CIHAUR GEULIS,-6.90168,107.62930
751,JAWA BARAT_BANDUNG_CIBEUNYING KALER_SUKALUYU,-6.89299,107.63438
752,JAWA BARAT_BANDUNG_GEDEBAGE_CIMINCRANG,-6.94604,107.70458


In [174]:
con.register('no_coordinate', no_coor_df[['unique_location','lat','long']])

In [175]:
con.sql('SELECT * FROM no_coordinate LIMIT 3')

┌───────────────────────────────────────┬────────────────────┬────────────────────┐
│            unique_location            │        lat         │        long        │
│                varchar                │       double       │       double       │
├───────────────────────────────────────┼────────────────────┼────────────────────┤
│ JAWA BARAT_INDRAMAYU_TERISI_CIBERENG  │ -6.454839999999933 │ 108.16669000000007 │
│ JAWA BARAT_INDRAMAYU_TERISI_CIKAWUNG  │ -6.607059999999933 │ 107.98109000000005 │
│ JAWA BARAT_INDRAMAYU_TERISI_JATIMULYA │ -6.501369999999952 │ 108.14761000000004 │
└───────────────────────────────────────┴────────────────────┴────────────────────┘

In [176]:
con.sql(
    """
    CREATE TABLE entity_with_coordinate2 AS
    SELECT * FROM entity_with_coordinate LEFT JOIN no_coordinate 
    ON entity_with_coordinate.unique_location = no_coordinate.unique_location
    """
    )

In [182]:
con.sql(
    """
    UPDATE entity_with_coordinate2
    SET lat = no_coordinate.lat, long = no_coordinate.long
    FROM no_coordinate
    WHERE entity_with_coordinate2.unique_location = no_coordinate.unique_location
    """
)

In [183]:
con.sql("SELECT unique_location,lat,long FROM entity_with_coordinate2 WHERE unique_location='JAWA BARAT_INDRAMAYU_TERISI_CIBERENG' LIMIT 3")

┌──────────────────────────────────────┬────────────────────┬────────────────────┐
│           unique_location            │        lat         │        long        │
│               varchar                │      varchar       │      varchar       │
├──────────────────────────────────────┼────────────────────┼────────────────────┤
│ JAWA BARAT_INDRAMAYU_TERISI_CIBERENG │ -6.454839999999933 │ 108.16669000000007 │
│ JAWA BARAT_INDRAMAYU_TERISI_CIBERENG │ -6.454839999999933 │ 108.16669000000007 │
│ JAWA BARAT_INDRAMAYU_TERISI_CIBERENG │ -6.454839999999933 │ 108.16669000000007 │
└──────────────────────────────────────┴────────────────────┴────────────────────┘

In [184]:
con.commit()

In [185]:
con.sql('SELECT COUNT(nik) FROM entity_with_coordinate2 WHERE lat IS NULL')

┌────────────┐
│ count(nik) │
│   int64    │
├────────────┤
│          0 │
└────────────┘

In [248]:
#preprocess
def finishing_coor_preprocess(coor:str) -> float:
    coor = coor.replace('.-', '.')
    coor = float(coor) # type: ignore
    return coor # type: ignore

# con.remove_function('finishing_coor_preprocess')

from duckdb.typing import FLOAT

con.create_function('finishing_coor_preprocess',finishing_coor_preprocess,[VARCHAR],FLOAT,exception_handling='return_null')


con.sql("ALTER TABLE entity_with_coordinate2 ADD COLUMN lat_clean FLOAT")
con.sql("ALTER TABLE entity_with_coordinate2 ADD COLUMN long_clean FLOAT")

con.sql("UPDATE entity_with_coordinate2 SET lat_clean = finishing_coor_preprocess(lat)")
con.sql("UPDATE entity_with_coordinate2 SET long_clean = finishing_coor_preprocess(long)")


In [247]:
vvv:str = '1'
float(vvv)

1.0

# Exploration
- asep V
- batak V
- astrologi V
- all name V
- cewe cowo V
- inisial r V
- cewe nama V
- jawa V
- nama kristen V
- produksi V
- trend nama V
- daerah pendatang V
- pergerakan

In [192]:
columns:pd.DataFrame = con.sql("SELECT column_name FROM information_schema.columns WHERE table_name = 'entity_with_coordinate2'").fetchdf()
print(columns['column_name'].to_list())



['nama', 'nik', 'tempatLahir', 'jenisKelamin', 'tps', 'provinsi', 'kab_kota', 'kecamatan', 'desa', 'newcomer', 'nama_sunda', 'nama_jawa', 'nama_batak', 'bintang', 'inisial_r', 'nama_kristen', 'umur', 'tahun_lahir', 'tanggal_produksi', 'hari_produksi', 'unique_location', 'unique_hash', 'lat', 'long', 'count', 'unique_location:1', 'lat:1', 'long:1']


In [223]:
# row total
con.sql("SELECT COUNT(nik) FROM entity_with_coordinate2")

┌────────────┐
│ count(nik) │
│   int64    │
├────────────┤
│   32049703 │
└────────────┘

### 1. Sundanese Name
- total orang dengan nama khas sunda
- top 10 nama sunda
- top 10 nama sunda di bandung
- kepadatan asep
- peta asep se jawa barat
- peta asep se bandung
- trend nama asep



#### total orang, top 10 nama sunda, top 10 per sample, kepadatan asep

In [198]:
# total orang dengan nama khas sunda
con.sql("SELECT COUNT(nik) FROM entity_with_coordinate2 WHERE nama_sunda IS NOT NULL")

┌────────────┐
│ count(nik) │
│   int64    │
├────────────┤
│    2665811 │
└────────────┘

In [224]:
2665811/32049703*100

8.317740105111113

In [225]:
# top 10 nama sunda jabar
print('JAWA BARAT')
con.sql("""
    SELECT nama_sunda, COUNT(nik) AS TOTAL 
    FROM entity_with_coordinate2 
    WHERE nama_sunda IS NOT NULL 
    GROUP BY nama_sunda 
    ORDER BY total DESC
    """)

JAWA BARAT


┌──────────────┬────────┐
│  nama_sunda  │ TOTAL  │
│   varchar    │ int64  │
├──────────────┼────────┤
│ asep         │ 293237 │
│ ade          │ 263750 │
│ agus         │ 227852 │
│ endang       │ 109907 │
│ dedi         │ 105186 │
│ wawan        │  96269 │
│ dadang       │  76152 │
│ maman        │  75704 │
│ cucu         │  72865 │
│ edi          │  71140 │
│  ·           │     ·  │
│  ·           │     ·  │
│  ·           │     ·  │
│ oded         │    256 │
│ sudarsana    │    130 │
│ nataprawira  │     94 │
│ surawijaya   │     75 │
│ markasih     │     63 │
│ sujoni       │     61 │
│ kamandaka    │     19 │
│ gugu         │     18 │
│ prawiraharja │     15 │
│ kertarajasa  │      3 │
├──────────────┴────────┤
│  101 rows (20 shown)  │
└───────────────────────┘

In [200]:
# top 10 nama sunda jabar
print()
con.sql("""
    SELECT nama_sunda, COUNT(nik) AS TOTAL 
    FROM entity_with_coordinate2 
    WHERE nama_sunda IS NOT NULL 
    GROUP BY nama_sunda 
    ORDER BY total DESC
    """)

┌──────────────┬────────┐
│  nama_sunda  │ TOTAL  │
│   varchar    │ int64  │
├──────────────┼────────┤
│ asep         │ 293237 │
│ ade          │ 263750 │
│ agus         │ 227852 │
│ endang       │ 109907 │
│ dedi         │ 105186 │
│ wawan        │  96269 │
│ dadang       │  76152 │
│ maman        │  75704 │
│ cucu         │  72865 │
│ edi          │  71140 │
│  ·           │     ·  │
│  ·           │     ·  │
│  ·           │     ·  │
│ oded         │    256 │
│ sudarsana    │    130 │
│ nataprawira  │     94 │
│ surawijaya   │     75 │
│ markasih     │     63 │
│ sujoni       │     61 │
│ kamandaka    │     19 │
│ gugu         │     18 │
│ prawiraharja │     15 │
│ kertarajasa  │      3 │
├──────────────┴────────┤
│  101 rows (20 shown)  │
└───────────────────────┘

In [226]:
# top 10 nama sunda bandung raya
# sumedang,tasikmalaya,sukabumi is null
# KOTA BANDUNG
print("KOTA BANDUNG")
con.sql("""
    SELECT nama_sunda, COUNT(nik) AS TOTAL 
    FROM entity_with_coordinate2 
    WHERE nama_sunda IS NOT NULL AND kab_kota='KOTA BANDUNG'
    GROUP BY nama_sunda 
    ORDER BY total DESC
    """)

KOTA BANDUNG


┌─────────────┬───────┐
│ nama_sunda  │ TOTAL │
│   varchar   │ int64 │
├─────────────┼───────┤
│ asep        │ 22398 │
│ agus        │ 16550 │
│ ade         │ 10556 │
│ dedi        │  6770 │
│ endang      │  6374 │
│ dadang      │  6033 │
│ wawan       │  5785 │
│ kurniawan   │  5070 │
│ cucu        │  4924 │
│ supriatna   │  4380 │
│  ·          │     · │
│  ·          │     · │
│  ·          │     · │
│ aom         │    37 │
│ jalu        │    34 │
│ barna       │    28 │
│ ece         │    26 │
│ oded        │    17 │
│ nataprawira │    16 │
│ sudarsana   │     8 │
│ surawijaya  │     5 │
│ sujoni      │     2 │
│ gugu        │     1 │
├─────────────┴───────┤
│ 97 rows (20 shown)  │
└─────────────────────┘

In [227]:
# KOTA CIMAHI
print("KOTA CIMAHI")
con.sql("""
    SELECT nama_sunda, COUNT(nik) AS TOTAL 
    FROM entity_with_coordinate2 
    WHERE nama_sunda IS NOT NULL AND kab_kota='KOTA CIMAHI'
    GROUP BY nama_sunda 
    ORDER BY total DESC
    """) 

KOTA CIMAHI


┌─────────────┬───────┐
│ nama_sunda  │ TOTAL │
│   varchar   │ int64 │
├─────────────┼───────┤
│ asep        │  5386 │
│ agus        │  4054 │
│ ade         │  2617 │
│ dedi        │  1552 │
│ endang      │  1481 │
│ dadang      │  1343 │
│ wawan       │  1306 │
│ cucu        │  1239 │
│ supriatna   │  1047 │
│ kurniawan   │  1005 │
│  ·          │     · │
│  ·          │     · │
│  ·          │     · │
│ ece         │     7 │
│ barna       │     7 │
│ jalu        │     5 │
│ aom         │     3 │
│ wisesa      │     3 │
│ kertarajasa │     1 │
│ surawijaya  │     1 │
│ sujoni      │     1 │
│ oded        │     1 │
│ kamandaka   │     1 │
├─────────────┴───────┤
│ 97 rows (20 shown)  │
└─────────────────────┘

In [228]:
# BANDUNG BARAT
print("BANDUNG BARAT")
con.sql("""
    SELECT nama_sunda, COUNT(nik) AS TOTAL 
    FROM entity_with_coordinate2 
    WHERE nama_sunda IS NOT NULL AND kab_kota='BANDUNG BARAT'
    GROUP BY nama_sunda 
    ORDER BY total DESC
    """) 

BANDUNG BARAT


┌─────────────┬───────┐
│ nama_sunda  │ TOTAL │
│   varchar   │ int64 │
├─────────────┼───────┤
│ asep        │ 27199 │
│ ade         │ 17961 │
│ agus        │ 14867 │
│ cucu        │  8469 │
│ wawan       │  7435 │
│ dedi        │  7278 │
│ dadang      │  6676 │
│ endang      │  5941 │
│ jajang      │  5670 │
│ supriatna   │  5150 │
│   ·         │     · │
│   ·         │     · │
│   ·         │     · │
│ ageng       │    25 │
│ sutresna    │    24 │
│ agah        │    16 │
│ oded        │    15 │
│ wisesa      │    13 │
│ sudarsana   │    10 │
│ aom         │     8 │
│ jalu        │     7 │
│ nataprawira │     7 │
│ gugu        │     2 │
├─────────────┴───────┤
│ 95 rows (20 shown)  │
└─────────────────────┘

In [229]:
# BANDUNG
print("BANDUNG")
con.sql("""
    SELECT nama_sunda, COUNT(nik) AS TOTAL 
    FROM entity_with_coordinate2 
    WHERE nama_sunda IS NOT NULL AND kab_kota='BANDUNG'
    GROUP BY nama_sunda 
    ORDER BY total DESC
    """) 

BANDUNG


┌─────────────┬───────┐
│ nama_sunda  │ TOTAL │
│   varchar   │ int64 │
├─────────────┼───────┤
│ asep        │ 49885 │
│ agus        │ 32677 │
│ ade         │ 31400 │
│ wawan       │ 15555 │
│ dadang      │ 15211 │
│ cucu        │ 14855 │
│ dedi        │ 14696 │
│ ayi         │ 14030 │
│ jajang      │ 12621 │
│ endang      │ 11536 │
│   ·         │     · │
│   ·         │     · │
│   ·         │     · │
│ karta       │    55 │
│ aom         │    53 │
│ sutresna    │    34 │
│ wisesa      │    19 │
│ jalu        │    18 │
│ sudarsana   │    11 │
│ nataprawira │     8 │
│ surawijaya  │     7 │
│ sujoni      │     2 │
│ gugu        │     1 │
├─────────────┴───────┤
│ 97 rows (20 shown)  │
└─────────────────────┘

In [230]:
# BOGOR
print("BOGOR")
con.sql("""
    SELECT nama_sunda, COUNT(nik) AS TOTAL 
    FROM entity_with_coordinate2 
    WHERE nama_sunda IS NOT NULL AND kab_kota='BOGOR'
    GROUP BY nama_sunda 
    ORDER BY total DESC
    """) 

BOGOR


┌──────────────┬───────┐
│  nama_sunda  │ TOTAL │
│   varchar    │ int64 │
├──────────────┼───────┤
│ asep         │ 30093 │
│ ade          │ 25984 │
│ agus         │ 25045 │
│ endang       │ 14323 │
│ dedi         │ 10890 │
│ edi          │  9761 │
│ wawan        │  7622 │
│ kurniawan    │  7615 │
│ junaedi      │  7351 │
│ hendra       │  7348 │
│   ·          │     · │
│   ·          │     · │
│   ·          │     · │
│ surawijaya   │    13 │
│ henhen       │    13 │
│ nataprawira  │     9 │
│ markasih     │     8 │
│ sujoni       │     5 │
│ sudarsana    │     4 │
│ kamandaka    │     3 │
│ gugu         │     1 │
│ prawiraharja │     1 │
│ oded         │     1 │
├──────────────┴───────┤
│ 100 rows (20 shown)  │
└──────────────────────┘

In [208]:
con.sql("SELECT COUNT(nik) FROM entity_with_coordinate2 WHERE nama_sunda IS NOT NULL AND kab_kota='BANDUNG'")

┌────────────┐
│ count(nik) │
│   int64    │
├────────────┤
│     406422 │
└────────────┘

In [213]:
con.sql("SELECT COUNT(nik) FROM entity_with_coordinate2 WHERE nama_sunda IS NOT NULL AND kab_kota ILIKE '%SUMEDANG%'")

┌────────────┐
│ count(nik) │
│   int64    │
├────────────┤
│          0 │
└────────────┘

In [233]:
kab_kota_sunda = con.sql("SELECT COUNT(nik) AS TOTAL, kab_kota FROM entity_with_coordinate2 GROUP BY kab_kota").fetchdf()
kab_kota_sunda
#sumedang,tasikmalaya,sukabumi

,TOTAL,kab_kota
0,1092869,CIAMIS
1,2073952,CIANJUR
2,1273891,SUBANG
3,2076923,GARUT
4,418488,PANGANDARAN
5,2046983,BEKASI
6,4144259,BOGOR
7,782080,PURWAKARTA
8,1553379,INDRAMAYU
9,1501109,BANDUNG BARAT


In [234]:
kab_kota_sunda_asep = con.sql("SELECT COUNT(nik) AS TOTAL_ASEP , kab_kota FROM entity_with_coordinate2 WHERE nama_sunda='asep' GROUP BY kab_kota").fetchdf()
kab_kota_sunda_asep

,TOTAL_ASEP,kab_kota
0,5742,KOTA TASIKMALAYA
1,9737,MAJALENGKA
2,22398,KOTA BANDUNG
3,30093,BOGOR
4,3066,KOTA BEKASI
5,5386,KOTA CIMAHI
6,3013,KOTA SUKABUMI
7,7902,BEKASI
8,5773,CIREBON
9,9459,PURWAKARTA


In [243]:
# kepadatan asep
kepadatan_asep = pd.merge(left=kab_kota_sunda,right=kab_kota_sunda_asep,how='inner',on=['kab_kota'])
kepadatan_asep['%'] = kepadatan_asep['TOTAL_ASEP'] / kepadatan_asep['TOTAL'] * 100
kepadatan_asep[['kab_kota','TOTAL','TOTAL_ASEP','%']].sort_values('%',ascending=False).reset_index(drop=True)

,kab_kota,TOTAL,TOTAL_ASEP,%
0,BANDUNG BARAT,1501109,27199,1.811927
1,BANDUNG,2855612,49885,1.746911
2,KOTA CIMAHI,364980,5386,1.475697
3,GARUT,2076923,30594,1.473044
4,CIANJUR,2073952,28522,1.375249
5,KOTA BANDUNG,1659017,22398,1.350077
6,KOTA SUKABUMI,225349,3013,1.337037
7,KOTA TASIKMALAYA,473633,5742,1.212331
8,PURWAKARTA,782080,9459,1.209467
9,SUBANG,1273891,11762,0.923313


#### peta asep

In [259]:
print("map")
map_asep = con.sql("""
    SELECT provinsi, kab_kota, kecamatan, desa, COUNT(nik) AS TOTAL, MIN(lat_clean) AS lat, MAX(long_clean) AS long
    FROM entity_with_coordinate2 
    WHERE nama_sunda = 'asep'
    GROUP BY provinsi, kab_kota, kecamatan, desa 
    ORDER BY TOTAL DESC
    """).fetchdf()
map_asep

map


,provinsi,kab_kota,kecamatan,desa,TOTAL,lat,long
0,JAWA BARAT,BANDUNG,BALEENDAH,BALEENDAH,745,-7.010890,107.632919
1,JAWA BARAT,BANDUNG,PANGALENGAN,SUKAMANAH,738,-7.206000,107.591003
2,JAWA BARAT,BANDUNG,PANGALENGAN,PANGALENGAN,698,-7.233000,107.554001
3,JAWA BARAT,KOTA CIMAHI,CIMAHI SELATAN,CIBEUREUM,683,-6.912120,107.560417
4,JAWA BARAT,KOTA CIMAHI,CIMAHI SELATAN,MELONG,635,-6.921610,107.557739
...,...,...,...,...,...,...,...
4930,JAWA BARAT,INDRAMAYU,LELEA,TEMPEL,1,-6.470878,108.209465
4931,JAWA BARAT,INDRAMAYU,LOHBENER,LANJAN,1,-7.257779,111.873985
4932,JAWA BARAT,KUNINGAN,CILEBAK,CILIMUSARI,1,-7.692280,108.354797
4933,JAWA BARAT,CIREBON,GEGESIK,KEDUNGDALEM,1,-6.283500,108.283501


In [254]:
!pip install nbformat>=4.2.0


[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [252]:
!pip install plotly
import plotly.express as px

     -------------------------------------- 15.3/15.3 MB 982.6 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [264]:
fig = px.density_mapbox(map_asep, lat='lat', lon='long', z='TOTAL',radius=10,opacity=0.75,
                        mapbox_style="stamen-terrain")
 
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'TOTAL=%{z}<br>lat=%{lat}<br>long=%{lon}<extra></extra>',
              'lat': array([-7.01089  , -7.206    , -7.233    , ..., -7.69228  , -6.2835   ,
                            -6.2417245], dtype=float32),
              'lon': array([107.63292 , 107.591   , 107.554   , ..., 108.3548  , 108.2835  ,
                            108.134415], dtype=float32),
              'name': '',
              'opacity': 0.75,
              'radius': 10,
              'subplot': 'mapbox',
              'type': 'densitymapbox',
              'z': array([745, 738, 698, ...,   1,   1,   1], dtype=int64)}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'TOTAL'}},
                             'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'legend': {'tracegroupgap': 0},
               'mapbox': {'center': {'lat': -6.2916317, 'lon': 113.60794},
                          'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                          'style': 'stamen-terrain',
                          'zoom': 8},
               'margin': {'t': 60},
               'template': '...'}
})

In [276]:
coor_outlier = map_asep[
    (map_asep['lat']<-7.9) |
    (map_asep['lat']>-6) |
    (map_asep['long']<105) |
    (map_asep['long']>109)
    ]
coor_outlier['unique_place'] = coor_outlier.provinsi + '_' + coor_outlier.kab_kota + '_' + coor_outlier.kecamatan + '_' + coor_outlier.desa
coor_outlier['new_coor'] = coor_outlier.unique_place.apply(scrap_coordinate)


C:\Users\yosia\AppData\Local\Temp\ipykernel_14788\1968951616.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yosia\AppData\Local\Temp\ipykernel_14788\1968951616.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [280]:
coor_outlier['lat'] = coor_outlier['new_coor'].apply(lambda x: x[0])
coor_outlier['long'] = coor_outlier['new_coor'].apply(lambda x: x[1])
coor_outlier

C:\Users\yosia\AppData\Local\Temp\ipykernel_14788\1857651357.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yosia\AppData\Local\Temp\ipykernel_14788\1857651357.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,provinsi,kab_kota,kecamatan,desa,TOTAL,lat,long,unique_place,new_coor
7,JAWA BARAT,BANDUNG,MAJALAYA,SUKAMAJU,554,-7.05177,107.75618,JAWA BARAT_BANDUNG_MAJALAYA_SUKAMAJU,"(-7.051769999999976, 107.75618000000009, Sukam..."
13,JAWA BARAT,CIANJUR,CIRANJANG,CIRANJANG,476,-6.80765,107.27754,JAWA BARAT_CIANJUR_CIRANJANG_CIRANJANG,"(-6.807649999999967, 107.27754000000004, Ciran..."
14,JAWA BARAT,BANDUNG,MAJALAYA,BIRU,472,-7.05323,107.72817,JAWA BARAT_BANDUNG_MAJALAYA_BIRU,"(-7.053229999999928, 107.72817000000003, Biru,..."
20,JAWA BARAT,BANDUNG BARAT,CIPATAT,GUNUNGMASIGIT,450,-6.83242,107.43895,JAWA BARAT_BANDUNG BARAT_CIPATAT_GUNUNGMASIGIT,"(-6.832419999999956, 107.43895000000003, Gunun..."
21,JAWA BARAT,BANDUNG,MAJALAYA,PADAMULYA,444,-7.05619,107.74966,JAWA BARAT_BANDUNG_MAJALAYA_PADAMULYA,"(-7.056189999999958, 107.74966000000006, Padam..."
...,...,...,...,...,...,...,...,...,...
4905,JAWA BARAT,KUNINGAN,KADUGEDE,MARGABAKTI,2,-7.04270,108.44570,JAWA BARAT_KUNINGAN_KADUGEDE_MARGABAKTI,"(-7.042699999999968, 108.44570000000004, Marga..."
4907,JAWA BARAT,CIREBON,GEGESIK,JAGAPURA KIDUL,2,-6.55353,108.43153,JAWA BARAT_CIREBON_GEGESIK_JAGAPURA KIDUL,"(-6.5535299999999665, 108.43153000000007, Jaga..."
4920,JAWA BARAT,KUNINGAN,MANDIRANCAN,NANGGERANGJAYA,1,-6.83017,108.48083,JAWA BARAT_KUNINGAN_MANDIRANCAN_NANGGERANGJAYA,"(-6.830169999999953, 108.48083000000008, Mandi..."
4928,JAWA BARAT,KUNINGAN,LURAGUNG,SINDANGSUKA,1,-7.01586,108.62962,JAWA BARAT_KUNINGAN_LURAGUNG_SINDANGSUKA,"(-7.015859999999975, 108.62962000000005, Sinda..."


In [281]:
map_asep['unique_place'] = coor_outlier.provinsi + '_' + coor_outlier.kab_kota + '_' + coor_outlier.kecamatan + '_' + coor_outlier.desa


In [290]:
map_asep_clean = pd.merge(coor_outlier[['unique_place','new_coor']], map_asep,how='outer',on='unique_place')
# map_asep_clean['lat'] = map_asep_clean.apply(lambda x:x['new_coor'][0] if x['new_coor'] else x['lat'], axis=1)
map_asep_clean['lat'] = map_asep_clean.apply(lambda x:x['new_coor'][0] if type(x['new_coor']) == tuple else x['lat'], axis=1)
map_asep_clean['long'] = map_asep_clean.apply(lambda x:x['new_coor'][1] if type(x['new_coor']) == tuple else x['long'], axis=1)
map_asep_clean = map_asep_clean[['provinsi','kab_kota','kecamatan','desa','TOTAL','lat','long']]
map_asep_clean

,provinsi,kab_kota,kecamatan,desa,TOTAL,lat,long
0,JAWA BARAT,BANDUNG,MAJALAYA,SUKAMAJU,554,-7.051770,107.756180
1,JAWA BARAT,CIANJUR,CIRANJANG,CIRANJANG,476,-6.807650,107.277540
2,JAWA BARAT,BANDUNG,MAJALAYA,BIRU,472,-7.053230,107.728170
3,JAWA BARAT,BANDUNG BARAT,CIPATAT,GUNUNGMASIGIT,450,-6.832420,107.438950
4,JAWA BARAT,BANDUNG,MAJALAYA,PADAMULYA,444,-7.056190,107.749660
...,...,...,...,...,...,...,...
4930,JAWA BARAT,CIREBON,SURANENGGALA,SURANENGGALA KIDUL,1,-6.375200,108.274200
4931,JAWA BARAT,INDRAMAYU,LELEA,TEMPEL,1,-6.470878,108.209465
4932,JAWA BARAT,KUNINGAN,CILEBAK,CILIMUSARI,1,-7.692280,108.354797
4933,JAWA BARAT,CIREBON,GEGESIK,KEDUNGDALEM,1,-6.283500,108.283501


In [283]:
map_asep

,provinsi,kab_kota,kecamatan,desa,TOTAL,lat,long,unique_place
0,JAWA BARAT,BANDUNG,BALEENDAH,BALEENDAH,745,-7.010890,107.632919,NaN
1,JAWA BARAT,BANDUNG,PANGALENGAN,SUKAMANAH,738,-7.206000,107.591003,NaN
2,JAWA BARAT,BANDUNG,PANGALENGAN,PANGALENGAN,698,-7.233000,107.554001,NaN
3,JAWA BARAT,KOTA CIMAHI,CIMAHI SELATAN,CIBEUREUM,683,-6.912120,107.560417,NaN
4,JAWA BARAT,KOTA CIMAHI,CIMAHI SELATAN,MELONG,635,-6.921610,107.557739,NaN
...,...,...,...,...,...,...,...,...
4930,JAWA BARAT,INDRAMAYU,LELEA,TEMPEL,1,-6.470878,108.209465,NaN
4931,JAWA BARAT,INDRAMAYU,LOHBENER,LANJAN,1,-7.257779,111.873985,JAWA BARAT_INDRAMAYU_LOHBENER_LANJAN
4932,JAWA BARAT,KUNINGAN,CILEBAK,CILIMUSARI,1,-7.692280,108.354797,NaN
4933,JAWA BARAT,CIREBON,GEGESIK,KEDUNGDALEM,1,-6.283500,108.283501,NaN


In [291]:
fig = px.density_mapbox(map_asep_clean, lat='lat', lon='long', z='TOTAL',radius=10,opacity=0.75,
                        mapbox_style="stamen-terrain")
 
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'TOTAL=%{z}<br>lat=%{lat}<br>long=%{lon}<extra></extra>',
              'lat': array([-7.05177   , -6.80765   , -7.05323   , ..., -7.69227982, -6.28350019,
                            -6.24172449]),
              'lon': array([107.75618   , 107.27754   , 107.72817   , ..., 108.35479736,
                            108.28350067, 108.13441467]),
              'name': '',
              'opacity': 0.75,
              'radius': 10,
              'subplot': 'mapbox',
              'type': 'densitymapbox',
              'z': array([554, 476, 472, ...,   1,   1,   1], dtype=int64)}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'TOTAL'}},
                             'colorscale': [[0.0, '#0d0887'], [0.1111111111111111,
                                            '#46039f'], [0.2222222222222222,
                                            '#7201a8'], [0.3333333333333333,
                                            '#9c179e'], [0.4444444444444444,
                                            '#bd3786'], [0.5555555555555556,
                                            '#d8576b'], [0.6666666666666666,
                                            '#ed7953'], [0.7777777777777778,
                                            '#fb9f3a'], [0.8888888888888888,
                                            '#fdca26'], [1.0, '#f0f921']]},
               'legend': {'tracegroupgap': 0},
               'mapbox': {'center': {'lat': -6.767403244430962, 'lon': 107.73034612795594},
                          'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                          'style': 'stamen-terrain',
                          'zoom': 8},
               'margin': {'t': 60},
               'template': '...'}
})

In [292]:
map_asep_clean.to_csv('clean_coordinate.csv')